In [10]:
import os
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv

In [11]:
load_dotenv(dotenv_path="/Users/priyal/Documents/honours/.env/key.env")
api_key = os.getenv("OPENAI_API_KEY")

In [12]:
client = OpenAI(api_key=api_key)

In [ ]:
FINE_TUNED_MODELS = {
    "Normal": "",
    "20-year-old female": "",
    "65-year-old female":"",
}

In [14]:
def chat_with_persona(question, persona):
    try:
        model_id = FINE_TUNED_MODELS[persona]
        stream = client.chat.completions.create(
            model=model_id,
            messages=[
                {"role": "system", "content": "You are an expert Ayurvedic assistant."},
                {"role": "user", "content": question},
            ],
            temperature=0.7,
            stream=True, 
        )
        output = ""
        for chunk in stream:
            if chunk.choices[0].delta.content:
                output += chunk.choices[0].delta.content
                yield output
    except Exception as e:
        yield f"❌ Error: {str(e)}"

In [18]:
def compare_all(question):
    responses = {}
    for persona, model_id in FINE_TUNED_MODELS.items():
        try:
            stream = client.chat.completions.create(
                model=model_id,
                messages=[
                    {
                        "role": "system",
                        "content": "You are an expert Ayurvedic assistant.",
                    },
                    {"role": "user", "content": question},
                ],
                temperature=0.7,
                stream=True,
            )
            output = ""
            for chunk in stream:
                if chunk.choices[0].delta.content:
                    output += chunk.choices[0].delta.content
            responses[persona] = output
        except Exception as e:
            responses[persona] = f"❌ Error: {str(e)}"
    return (
        responses["Normal"],
        responses["20-year-old female"],
        responses["65-year-old female"],
    )

In [19]:
with gr.Blocks(title="Ayurveda Chatbot Demo") as demo:
    gr.Markdown("##  Ayurveda Chatbot Demo")
    gr.Markdown(
        "Select a user persona and ask a question to see how your fine-tuned model responds."
    )

    persona = gr.Radio(
        choices=["Normal", "20-year-old female", "65-year-old female"],
        label="Select Persona",
        value="Normal",
    )

    question = gr.Textbox(
        label="Your Question", placeholder="e.g., What herbs help with digestion?"
    )
    output = gr.Textbox(label="Chatbot Response", lines=10)

    submit_btn = gr.Button("Get Answer")
    submit_btn.click(fn=chat_with_persona, inputs=[question, persona], outputs=output)

demo.launch()

* Running on local URL:  http://127.0.0.1:7862
* To create a public link, set `share=True` in `launch()`.
